In [15]:
import psycopg2
import pandas as pd
import boto3
from io import StringIO
import json
from sqlalchemy import create_engine
import urllib.parse

In [16]:
from configparser import ConfigParser

In [17]:
def credentialFile(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [18]:
def config(filename = "database.ini", section = "postgresql"):
    parser = ConfigParser()
    parser.read(filename)
    db = {}

    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception(f"section {section} not found in the {filename} file....")


    return db

In [19]:
def connect_to_db():
    try:
        params = config()
        connection = psycopg2.connect(**params)
        print("Database connect successfully....")
        return connection
    except Exception as e:
        print(f"Error while connecting to Database : {e}")
        return None

In [20]:
def fetch_records(connection):
    try:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM "SuperStoreSalingData"')
        records = cursor.fetchall()
        # return records

        columns = []
        for desc in cursor.description:
            columns.append(desc[0])

        df = pd.DataFrame(records, columns=columns)

        return df
        
    except Exception as e:
        print(f"Error while fetching records : {e}")
        return None
    finally:
        cursor.close()

In [21]:
def upload_dataframe_to_s3(dataframe, bucket, file_key, credentials_file):
    
    
    # Load AWS credentials
    credentials = credentialFile(credentials_file)

    # Initialize S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=credentials['aws_access_key_id'],
        aws_secret_access_key=credentials['aws_secret_access_key'],
        region_name=credentials['region_name']
    )

    try:
       
        # Convert the DataFrame to a CSV string
        
        csv_buffer = StringIO()
        dataframe.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)
        
        s3_client.put_object(Bucket=bucket, Body=csv_buffer.getvalue(), Key= file_key)
        print(f"DataFrame {file_key} uploaded to S3 successfully.")
        
    except Exception as e:
        print(f"Error uploading DataFrame to S3: {e}")

In [3]:
def main():
    connection = connect_to_db()

    if connection:
        try:
            df = fetch_records(connection)

            if df is not None: 
                bucket = 'bucket_name'
                file_key = 'file_name'
                credentials_file = 'credential_file_name'
                upload_dataframe_to_s3(df, bucket, file_key, credentials_file)

        except Exception as e:
            print(f"Error while fetching data : {e}")

        finally:
            connection.close()
            print("database connection closed....")

In [27]:
if __name__ == "__main__":
    main()

Database connect successfully....
database connection closed....
